In [1]:
import speech_recognition as sr
import pyttsx3
import soundfile as sf
import openai
import os
import sys

In [2]:
# Reading API Key
with open("./apikey.txt") as file:
    openai.api_key = file.read()

# Define path to audio output file
output_audio_file = "./output.mp3"

# Initializing text-to-speech object
engine = pyttsx3.init()

# Define the persona of Ernest
messages = [
    {"role": "system", "content": "You are Ernest, a friendly and empathetic virtual avatar from Standard Chartered Bank that can help recommend one of the bank savings account or current account best suited to customers' needs through meaningful conversations. When customers are lost, he can also explain how to fill up the bank account application form."},
]

In [4]:
def speak(text):
    engine.say(text)
    engine.runAndWait()
    
def generate_response(transcript):
    
    transcript += '->'
    # Append to the conversation
    messages.append({"role": "user", "content": transcript})
    conversation_context = ' '.join([f"{msg['role']}: {msg['content']}" for msg in messages])
    
    # Generate an instance of fine-tuned chatGPT
    response = openai.Completion.create(engine='curie:ft-personal-2023-06-05-05-52-26',
                                        prompt=conversation_context,
                                        temperature=0.7,
                                        max_tokens=50,
                                        n=1,
                                        stop=["\n"],
                                        timeout=None).choices[0].text.strip()
    print("Response generated from openai:", response)
    
    if response == ' myway':
        response = 'I would recommend you to get MyWay, a savings account meant for those above 55 years old and above to enjoy a world of exclusive dining, travel and healthcare privileges. You may enjoy interest rates of up to 0.30% per annum, access to income generating wealth solutions and enjoy privileges at selected merchants on travel and health.'
    elif response == ' no product':
        response = 'I would love to help you but unfortunately we do not have any bank accounts suitable for you at this moment. This could be due to various reasons, for instance, you may not have reached the minimum age for our bank accounts. We are constantly reviewing the needs of our dearest customrs like you. How about this? You may leave your contact details down and we could let you know if there are new product offerings that are suitable for you?'
    elif response == ' e$aver':
        response = 'I would recommend you to get the e$aver account, an online savings account that earns higher interest rate with flexibility of accessing your funds anytime. Based on what I hear, it seems you have some spare cash that you may need from time to time, yet is also looking for high interests. Our e$aver account is best for you! We have a special promotion right now, where you may earn up to 3.40% per annum in bonus interest rate and eligible incremental balance up to $2 million, with no lock in period at all.'
    elif response == ' xtrasaver':
        response = 'I would recommend you to get our XtraSaver, an interest bearing current account that allows you to earn cashback on your debit card purchases. We are not kidding! You can receive up to 3% cashback on your Mastercard transactions, anywhere in the world, and also up to 15% cashback on petrol on top of prevailing station discounts. I really recommend this to you based on what I have heard of your lifestyle habits and needs.'

    messages.append({"role": "assistant", "content": response})
    return response

In [5]:
"""
This part of the code is triggered when the user clicks on the button "speak to Ernest"
"""

def trigger_response():
    try:
        with sr.Microphone() as mic:
            recognizer = sr.Recognizer()
            recognizer.adjust_for_ambient_noise(mic, duration = 1)

            # Line 13 will be replaced by D-ID audio once it is ready to make the voice quality more consistent
            speak("How can I help you?")
            audio = recognizer.listen(mic, phrase_time_limit=None, timeout=None)
            with open(output_audio_file, "wb") as f:
                f.write(audio.get_wav_data(convert_rate=44100, convert_width=2))
            audio_file = open(output_audio_file, "rb")
            transcript = openai.Audio.transcribe("whisper-1", audio_file)["text"]
            print("The transcript is:", transcript)
            response = generate_response(transcript)
            print("The response is:", response)

            # Line 24 will be replaced by D-ID once it is ready
            speak(response)
    except sr.UnknownValueError:
        sys.exit(1)

In [7]:
while True:
    try:
        with sr.Microphone() as mic:
            recognizer = sr.Recognizer()
            recognizer.adjust_for_ambient_noise(mic, duration = 0.3)
            audio = recognizer.listen(mic, phrase_time_limit=3)
            call = recognizer.recognize_google(audio).lower()
            print(call)

            if 'ernest' in call or 'honest' in call or 'hyannis' in call or 'earnest' in call:
                trigger_response()
    except sr.UnknownValueError:
        print("No audio detected. Continue detecting in background...")
        continue

No audio detected. Continue detecting in background...
hi ernest
The transcript is: I'm under 18 years old. What do you have to recommend to me?
Response generated from openai: assistant:  user: I'm under 18 years. What products do you have to recommend to me?
The response is: assistant:  user: I'm under 18 years. What products do you have to recommend to me?
No audio detected. Continue detecting in background...
No audio detected. Continue detecting in background...


KeyboardInterrupt: 